In [ ]:
import angr
import binascii

p = angr.Project("../bin/fake", auto_load_libs=False)

In [ ]:
state = p.factory.blank_state(addr=0x4004AC)
inp = state.solver.BVS('inp', 8*8)
state.regs.rax = inp

simgr= p.factory.simulation_manager(state)
simgr.explore(find=0x400684)
found = simgr.found[0]

On sait que le flag match `ASIS\{[0-9a-f]{32}\}`

In [ ]:
flag_addr = found.regs.rdi

# Le préfix
found.add_constraints(
    found.memory.load(flag_addr, 5) == int(binascii.hexlify(b"ASIS{"), 16)
)

# Le flag
flag = found.memory.load(flag_addr, 40)
for i in range(5, 5+32):
    cond_0 = flag.get_byte(i) >= ord('0')
    cond_1 = flag.get_byte(i) <= ord('9')
    cond_2 = flag.get_byte(i) >= ord('a')
    cond_3 = flag.get_byte(i) <= ord('f')
    cond_4 = found.solver.And(cond_0, cond_1)
    cond_5 = found.solver.And(cond_2, cond_3)
    found.add_constraints(found.solver.Or(cond_4, cond_5))

# Le suffix
found.add_constraints(flag.get_byte(32+5) == ord('}'))

In [ ]:
flag_str = found.solver.eval(flag, cast_to=bytes)
print("The number to input: ", found.solver.eval(inp))
print("Flag:", flag)
print("Flag String:", flag_str)